In [2]:
%matplotlib inline

from __future__ import unicode_literals

import json
import logging
import math
import os
import re

import matplotlib.pyplot as plt
import numpy as np

In [3]:
class TrainLogLine(object):
    __slots__ = ('iteration', 'elapsed', 'test_loss', 'train_loss', 'best', 'best_iteration')

    def __init__(self, **kwargs):
        for key, value in kwargs.iteritems():
            setattr(self, key, value)

    def __unicode__(self):
        return (
            'Training batch {iteration}; ' +
            'Elapsed {elapsed}; ' +
            'loss: {test_loss} (train: {train_loss}); ' +
            'best: {best} ({best_iteration})'
        ).format(**self.to_dict())

    def __repr__(self):
        return self.__unicode__()

    def to_dict(self):
        return {
            key: getattr(self, key)
            for key in self.__slots__
            if hasattr(self, key)
        }


def yield_train_log_line(log_path):
    with open(log_path, 'r') as f:
        for line in f:
            if is_train_log_line(line):
                yield parse_line(line)


def parse_line(line):
    r = r'^.*Training batch ([0-9]+); Elapsed ([0-9]+)s; loss: ([0-9\.]+) \(train: ([0-9\.]+)\); best: ([0-9\.]+) \(([0-9]+)\).*$'
    m = re.match(r, line)

    if m is None:
        raise ValueError('No match for line {}'.format(line))

    return TrainLogLine(
        iteration=int(m.group(1)),
        elapsed=int(m.group(2)),
        test_loss=float(m.group(3)),
        train_loss=float(m.group(4)),
        best=float(m.group(5)),
        best_iteration=int(m.group(6)),
    )


def is_train_log_line(line):
    return re.search('Training batch', line) is not None

In [4]:
log_lines = [l for l in yield_train_log_line('/Users/srom/workspace/go/src/github.com/srom/chessbot/train3.log')]

In [5]:
len(log_lines)

84478

In [6]:
iter_min = 10000
x = range(0, len(log_lines))[iter_min:]
y = [l.test_loss for l in log_lines][iter_min:]
# plt.plot(x, y)